In [ ]:
%load_ext autoreload
%autoreload 2

https://soft.mines-paristech.fr/gstlearn/courses-latest/python/06a_Multivariate.html

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import gstools as gs

verbose = True
graphics = True
gl.OptCst.define(gl.ECst.NTCOL, 6)
gdoc.setNoScroll()

# Data

### Loading

In [ ]:
from bramm_data_analysis import loaders

# Moss
moss_data_path = Path("../data/Mines_2024.xlsx")
df_moss = loaders.from_moss_csv(moss_data_path).retrieve_df()
# RMQS
rmqs_data_path = Path("../data/RMQS.csv")
df_rmqs = loaders.from_rmqs_csv(rmqs_data_path).retrieve_df()

### Matching

See more in the [matching-related notebook](matching.ipynb).

In [ ]:
from bramm_data_analysis.matching import Matcher

data_matcher = Matcher(
    km_threshold=5,
    year_threshold=2006,
)
matched_df = data_matcher.match_rmqs_to_moss(
    df_moss,
    df_rmqs,
    radians=False,
)

print(
    f" Year Threshold : {data_matcher.year_threshold} \n",
    f"Distance Threshold : {data_matcher.km_threshold} km \n",
    f"Conserved : {matched_df.shape[0]} / {df_moss.shape[0]}",
)

# MultiVariate Analysis

### Variable Selection

In [ ]:
x1 = "longitude"
x2 = "latitude"
z1 = "copper"
z2 = "cu_tot_hf"

df = matched_df.filter(["longitude_rmqs", "latitude_rmqs", z1, z2]).rename(
    columns={
        "latitude_rmqs": "latitude",
        "longitude_rmqs": "longitude",
    },
)
df[df[z2] == "ND"] = np.nan
df[z2] = df[z2].astype("float")
df_rmqs[df_rmqs[z2] == "ND"] = np.nan
df_rmqs = df_rmqs[~df_rmqs[z2].isna()]
df_rmqs[z2] = df_rmqs[z2].astype("float")

In [ ]:
# Observations
observations = gl.Db_fromPanda(
    df.filter([x1, x2, z1, z2]).drop_duplicates([x1, x2])
)
observations.setLocators([x1, x2], gl.ELoc.X)
observations.setLocator(z1, gl.ELoc.Z)

# Grid
targets = gl.Db_fromPanda(
    df_rmqs.filter([x1, x2, z2]).drop_duplicates([x1, x2])
)
targets.setLocators([x1, x2], gl.ELoc.X)
targets.setLocator(z2, gl.ELoc.Z)

In [ ]:
ax = targets.plot(z2)

In [ ]:
gl.dbStatisticsPrint(
    db=observations,
    names=[z2, z1],
    opers=gl.EStatOption.fromKeys(["NUM"]),
    flagIso=False,
    title="Number of observations",
)

In [ ]:
gl.dbStatisticsPrint(
    db=observations,
    names=[z1, z2],
    opers=gl.EStatOption.fromKeys(["MEAN", "MINI", "MAXI"]),
    flagIso=False,
    title="Statistics of observations",
)

In [ ]:
gl.dbStatisticsPrint(
    db=targets,
    names=[z2],
    opers=gl.EStatOption.fromKeys(["MEAN", "MINI", "MAXI"]),
    flagIso=False,
    title="Statistics of target",
)

In [ ]:
gl.dbStatisticsPrint(
    db=observations,
    names=[z1, z2],
    opers=gl.EStatOption.fromKeys(["MEAN", "MINI", "MAXI"]),
    flagIso=True,
    title="Filtered statistics of observations",
)

In [ ]:
ax = targets.plot(z2)
ax = observations.plot(
    nameSize=z1,
    color="yellow",
    sizmin=0.1,
    sizmax=20,
)

In [ ]:
ax = gp.correlation(
    observations, namex=z2, namey=z1, asPoint=True, regrLine=True
)
ax.decoration(title=f"Correlation between {z1} and {z2}")

In [ ]:
varioparam = gl.VarioParam.createMultiple(ndir=2, npas=30, dpas=1)
vario_raw2dir = gl.Vario(varioparam)
err = vario_raw2dir.compute(observations)

In [ ]:
fitmod_raw = gl.Model()
err = fitmod_raw.fit(
    vario_raw2dir,
    types=[gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.CUBIC, gl.ECov.LINEAR],
)
fitmod_raw.setDriftIRF(0, 0)

In [ ]:
ax = gp.varmod(vario_raw2dir, fitmod_raw)
ax.decoration(
    title=f"Experimental and fitted variogram models - Raw {z2} observations",
    xlabel="Distance (km)",
    ylabel="Variogram",
)

In [ ]:
uniqueNeigh = gl.NeighUnique.create()

In [ ]:
err = gl.kriging(
    dbin=observations,
    dbout=targets,
    model=fitmod_raw,
    neigh=uniqueNeigh,
    namconv=gl.NamingConvention.create(prefix="OK"),
)

In [ ]:
ax = targets.plot("OK*estim")
ax = observations.plot(flagCst=True, color="black")
ax.decoration(title=f"{z1} - Ordinary Kriging")

In [ ]:
opers = gl.EStatOption.fromKeys(["NUM", "MINI", "MAXI", "MEAN", "STDV"])
gl.dbStatisticsPrint(
    targets,
    names=(["OK.T*"]),
    opers=opers,
    title="Statistics on the Ordinary Kriging:",
)

In [ ]:
## Compute cross-validation
err = gl.xvalid(
    observations,
    model=fitmod_raw,
    neigh=uniqueNeigh,
    namconv=gl.NamingConvention.create(prefix="CV_OK", flag_locator=False),
)

In [ ]:
mse = np.nanmean(np.square(observations.getColumn("CV_OK*esterr*")))
print("Mean squared cross-validation error:", round(mse, 3))

mse = np.nanmean(np.square(observations.getColumn("CV_OK*stderr*")))
print("Mean squared standardized error:", round(mse, 3))

In [ ]:
observations.setLocators(names=[z1, z2], locatorType=gl.ELoc.Z)
observations

In [ ]:
varioexp2var = gl.Vario.create(varioparam)
err = varioexp2var.compute(observations)
ax = gp.varmod(varioexp2var)

In [ ]:
fitmod2var = gl.Model()
err = fitmod2var.fit(
    varioexp2var,
    types=[gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.CUBIC, gl.ECov.LINEAR],
)
fitmod2var.setDriftIRF(0, 0)

In [ ]:
ax = gp.varmod(varioexp2var, fitmod2var, lw=2)
gp.decoration(ax, title=f"{z1} and {z2}")

In [ ]:
err = gl.kriging(
    dbin=observations,
    dbout=targets,
    model=fitmod2var,
    neigh=uniqueNeigh,
    namconv=gl.NamingConvention.create(prefix="COK"),
)

In [ ]:
ax = targets.plot(f"COK.{z1}*estim")
ax = observations.plot(flagCst=True, color="black")
ax.decoration(title=f"{z1} - CoKriging")

In [ ]:
opers = gl.EStatOption.fromKeys(["NUM", "MINI", "MAXI", "MEAN", "STDV"])
gl.dbStatisticsPrint(
    targets,
    names=(["COK.*"]),
    opers=opers,
    title="Statistics on the CoKriging predictions",
)

In [ ]:
ax = gp.correlation(
    targets,
    namex="OK.*estim",
    namey="COK.copper.estim",
    bissLine=True,
    bins=100,
    cmin=1,
)
ax.decoration(xlabel="Ordinary Kriging", ylabel="Ordinary CoKriging")